In [1]:
# -*- coding: utf-8 -*- 

from pykrx import stock
import parmap
import pandas as pd
from tabulate import tabulate
import Utils
import pandas_ta as ta


def get_result(ticker):
    try:
        name = stock.get_market_ticker_name(ticker)
        if '스팩' in name:
            return 'pass'
        df = stock.get_market_ohlcv_by_date("20000101", Today, ticker)
        temp_list = df['종가'].tolist()
        if len(temp_list) < 270:
            return 'pass'
        sorted_list = sorted(temp_list)
        if temp_list[-1] == sorted_list[-1] and sorted_list[-2] not in temp_list[-30:]:
            cap = int(stock.get_market_cap("20220101", Today, ticker)['시가총액'].tail(1))
            cap = 0
            size = get_position_size(df)
            return [name, round(max(temp_list[-250:])/min(temp_list[-250:]), 1), cap, size]
    except:
        print('err', end='')
    return 'pass'


def print_result(input_list):
    temp_list = [[],[],[],[],[]]

    for idx, item in enumerate(input_list):
        temp_list[0].append(item[0])
        temp_list[1].append(item[1])
        temp_list[2].append(item[2])
        temp_list[3].append(item[3])

    df = pd.DataFrame({'name':temp_list[0], 'max/min':temp_list[1], 'cap':temp_list[2], 'size':temp_list[3]})
    df['R_gap'] = df['max/min'].rank(method='min')
    df['R_cap'] = df['cap'].rank(method='min', ascending=False)
    df['gap+cap'] = df['R_gap'] + df['R_cap']

    return tabulate(df, headers='keys', tablefmt='psql')


def get_target_list_sub(ticker):
    try:
        name = stock.get_market_ticker_name(ticker)
        if '스팩' in name:
            return None
        df = stock.get_market_ohlcv_by_date('20000301', Today, ticker)
        temp_list = df['종가'].tolist()
        sorted_list = sorted(temp_list)
        if temp_list[-1] > temp_list[-2] and temp_list[-1] * 1.3 > sorted_list[-1]:
            return ticker
    except:
        return None


def get_target_list():
    konex = stock.get_market_ticker_list(market="KONEX")
    tickers = list(set(stock.get_market_ticker_list(market="ALL")) - set(konex))
    # result_ = parmap.map(get_result, tickers , pm_pbar=True, pm_processes=2)
    result_ = parmap.map(get_target_list_sub, tickers , pm_pbar=True, pm_processes=2)
    while(1):
        try:
            result_.remove(None)
        except:
            break
    return result_


def get_close_price(ticker):
    name = stock.get_market_ticker_name(ticker)
    df = stock.get_market_ohlcv_by_date('20000301', Today, ticker)
    temp_list = df['종가'].tolist()
    temp_list.append(ticker)
    return temp_list




if __name__ == '__main__':
    global Today
    Today = Utils.get_time()[0]
    # Today = '20210710'

    

    
    konex = stock.get_market_ticker_list(market="KONEX")
    tickers = list(set(stock.get_market_ticker_list(market="ALL")) - set(konex))
    result = {}

    result_ = parmap.map(get_close_price, tickers , pm_pbar=True, pm_processes=2)

    for data in result_:
        result[data[-1]] = data[:-1]

    print(result.keys())
    
#####################################################

import pickle


file_name = 'stock_close_data.pickle'


with open(file_name,"wb") as fw:
    pickle.dump(result, fw)
    






  0%|          | 0/2496 [00:00<?, ?it/s]

dict_keys(['065420', '900070', '131400', '034940', '073640', '317530', '389030', '083420', '159910', '123010', '217600', '086820', '018470', '239340', '222080', '192820', '122450', '306040', '079550', '101330', '032280', '009470', '158430', '101360', '066670', '036690', '322780', '126560', '348370', '008355', '041920', '304840', '010280', '010050', '068400', '011700', '204630', '170920', '022220', '060230', '298690', '012790', '005935', '058470', '068330', '067390', '000880', '058970', '000590', '000810', '073190', '036090', '069730', '002900', '067160', '001630', '112610', '005740', '348210', '331520', '008700', '097870', '026150', '007660', '003610', '009900', '047080', '086960', '178920', '001360', '396300', '003230', '005680', '060240', '322510', '272110', '025770', '120110', '060480', '007980', '281740', '160980', '136490', '226440', '900280', '053260', '004255', '002020', '001200', '004920', '090710', '307870', '316140', '003670', '003000', '106190', '003200', '052670', '096240',

In [3]:
# -*- coding: utf-8 -*- 

from pykrx import stock
import parmap
import pandas as pd
from tabulate import tabulate
import Utils
import pandas_ta as ta
import gzip




def get_ohlcvs(ticker):
    result = []
    df = stock.get_market_ohlcv_by_date('20000301', Today, ticker)
    #temp_time = list(df.index)
    temp_open = df['시가'].tolist()
    temp_high = df['고가'].tolist()
    temp_low = df['저가'].tolist()
    temp_close = df['종가'].tolist()
    for i in range(len(temp_open)):
        # result.append([temp_time[i],temp_open[i], temp_high[i], temp_low[i], temp_close[i]])
        result.append([temp_open[i], temp_high[i], temp_low[i], temp_close[i]])
    result.append(ticker)
    return result




if __name__ == '__main__':
    global Today
    Today = Utils.get_time()[0]
    # Today = '20210710'

    

    
    konex = stock.get_market_ticker_list(market="KONEX")
    tickers = list(set(stock.get_market_ticker_list(market="ALL")) - set(konex))
    result = {}


    result_ = parmap.map(get_ohlcvs, tickers , pm_pbar=True, pm_processes=2)

    for data in result_:
        result[data[-1]] = data[:-1]

    
#####################################################

import pickle


file_name = 'stock_ohlcv_data.pickle'


with gzip.open(file_name,"wb") as fw:
    pickle.dump(result, fw)
    






  0%|          | 0/2496 [00:00<?, ?it/s]

In [2]:
import pickle

with open("stock_close_data.pickle","rb") as fr:
    result = pickle.load(fr)






In [12]:


ticker_list = list(result.keys())
#print(result[ticker_list[0]])
max = 0 
name = [] 
for i in range(len(ticker_list)): 
    length_of_close = len(result[ticker_list[i]]) #print(length_of_close)

if max < length_of_close:
    max = length_of_close
    name.append(ticker_list[i])
#print('############################################') #print(max) #print(name[-1])

ticker_name = ticker_list[1]
# ticker_name = ''

print(ticker_name)

ticker_close = result[ticker_name] 


sorted_close = sorted(ticker_close)

buy_sell_log = []

for i in range(len(ticker_close)): 
    if i < 365: 
        continue
    
    close_sample = ticker_close[:i]
    # sorted_sample = sorted(close_sample)
    

    
    if close_sample[-1] == sorted_sample[-1] and sorted_sample[-2] not in close_sample[-30:]:
    # if close_sample[-1] == sorted_sample[-1]:
        entry_price = ticker_close[i] 
        exit_price = ticker_close[i+2]
        earning = (exit_price / entry_price - 1) * 100
        buy_sell_log.append(earning)

        
import Utils
performance = Utils.Performance()

performance.calculating(buy_sell_log)




모다이노칩
